In [ ]:
%matplotlib inline

In [ ]:
import os
from casatasks import split, listobs

# overwrite output data if it exists
os.system('rm -rf twhya_smoothed.ms')
os.system('rm -f twhya_smoothed.listobs.txt')

split(vis='sis14_twhya_calibrated_flagged.ms', field='5', width='8', outputvis='twhya_smoothed.ms', datacolumn='data')

listobs('twhya_smoothed.ms', verbose=True, listfile='twhya_smoothed.listobs.txt')

In [ ]:
import priism.alma

In [ ]:
worker = priism.alma.AlmaSparseModelingImager(solver='mfista_nufft')

In [ ]:
worker.selectdata(vis='twhya_smoothed.ms',
                  spw='0:24',
                  intent='OBSERVE_TARGET#ON_SOURCE',
                  datacolumn='data')

In [ ]:
worker.defineimage(imsize=[250, 250],
                   cell=['0.08arcsec'],
                   phasecenter='0',  # TW Hya
                   nchan=1,
                   start=0,
                   width=1)

In [ ]:
%%timeit -n 1 -r 1

worker.readvis()

In [ ]:
%%timeit -n 1 -r 1

# 4min 30s ± 8.57 s per loop (mean ± std. dev. of 7 runs, 1 loop each)
worker.solve(l1=1e0, ltsv=1e8, maxiter=1000)

In [ ]:
worker.exportimage(imagename='myimage.fits', overwrite=True)

In [ ]:
# CV elapsed 6905.8 sec
L1_list = [1e2, 1e4, 1e6]
Ltsv_list = [1e6, 1e8, 1e10]
num_fold = 4

worker.crossvalidation(L1_list, Ltsv_list, num_fold=num_fold,
                       imageprefix='mycvresult', imagepolicy='full',
                       summarize=True, figfile='cvresult.png',
                       datafile='cvresult.dat', maxiter=1000)
